Task_1_explo
===

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import neurokit2 as nk

In [2]:
%matplotlib inline
import torch
import torch.nn.functional as F

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [4]:
import param
import panel as pn
pn.extension()
import datetime as dt
import importlib
from importlib import reload
from tqdm import tqdm

In [5]:
import graph as gr
import params as par
import py_lib as lib
reload(gr)
reload(par)
reload(lib)
import graph as gr
import params as par
import py_lib as lib

In [6]:
format_dict = par.create_format_dict()
format_dict

{'Age': '{:.0f}',
 'RestingBP': '{:.0f}',
 'Cholesterol': '{:.0f}',
 'MaxHR': '{:.0f}',
 'FastingBS': '{:.1f}',
 'HeartDisease': '{:.1f}',
 'Oldpeak': '{:.2f}',
 '0': '{:.0%}',
 '1': '{:.0%}',
 'Accuracy': '{:.1%}',
 'Balanced Accuracy': '{:.1%}',
 'F1 Score': '{:.1%}'}

In [7]:
class NotebookParams(param.Parameterized):
    lambda_multiple          = param.Number(1, bounds=(1,10), doc="lambda Lasso Multiple of 10^x")
    lambda_power             = param.Integer(default=2, bounds=(1,7),doc="lambda Lasso exponent of 10^x")
    lr_multiple              = param.Number(5, bounds=(1,10), doc="Learning Rate Multiple of 10^x")
    lr_power                 = param.Integer(default=3, bounds=(1,9),doc="Learning Rate exponent of 10^x")
    epochs                   = param.Integer(default=2000 ,bounds=(1,2000))
    data_set                 = param.ObjectSelector(default="test",objects=['train', 'validation', 'train + validation', 'test'])
    stand_features           = param.Boolean(True, doc="Whether we want to standardize numerical features")
    use_lasso                = param.Boolean(True)
    use_focal_loss           = param.Boolean(True)
    iter_array               = param.Integer(default=5, bounds=(1,20),doc="How often to save loss and training data, in epochs")
    iter_printing            = param.Integer(default=5, bounds=(1,100),doc="How many epochs between printing")
    add_one_hot_encoded      = param.Boolean(True, doc="Whether to add one hot encoded columns from categorical variables")
    test_size                = param.Number(0.2, bounds=(0.1,0.9), doc="Test size for train_test_split")
    random_state_test_train_split = param.Integer(42)
base = NotebookParams()

# Import Data

In [8]:
# df_heart = pd.read_csv('../data_proj2/ptbdb_train.csv')
df_heart_test = pd.read_csv('../data_proj2/ptbdb_test.csv', header=None)
df_heart_train = pd.read_csv('../data_proj2/ptbdb_train.csv', header=None)

In [11]:
df_heart_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.730599,0.472859,0.139526,0.000000,0.020507,0.112183,0.162847,0.180941,0.192199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.000000,0.152446,0.011377,0.308305,0.153584,0.096701,0.083049,0.086462,0.105802,0.071672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.975917,0.693234,0.241399,0.107225,0.098050,0.072248,0.062500,0.054472,0.054472,0.045872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.952505,0.587508,0.115810,0.072869,0.003904,0.025374,0.020169,0.020169,0.021470,0.014314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.994152,0.525666,0.248213,0.070825,0.152047,0.258609,0.315789,0.346979,0.352827,0.348928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
df_heart_train.columns

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187],
      dtype='int64', length=188)

In [10]:
df_heart_train.describe().style.format(format_dict)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187
count,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000,11641.000000
mean,0.976530,0.721439,0.402820,0.242706,0.206556,0.215723,0.221038,0.223554,0.226485,0.228690,0.231415,0.233240,0.235998,0.239128,0.241565,0.245459,0.248385,0.251952,0.256260,0.259369,0.263286,0.268197,0.275596,0.285904,0.296167,0.307767,0.317917,0.327244,0.337096,0.344853,0.351245,0.354447,0.355006,0.352592,0.345687,0.337693,0.326263,0.314676,0.302928,0.291419,0.283096,0.275565,0.269475,0.264432,0.261152,0.260856,0.261026,0.261672,0.262772,0.262046,0.262260,0.261307,0.259682,0.258642,0.256207,0.255477,0.252548,0.251015,0.250932,0.251319,0.252904,0.251834,0.251114,0.251032,0.249634,0.250351,0.251658,0.256658,0.257361,0.253206,0.253137,0.254715,0.250684,0.242211,0.235837,0.234976,0.234018,0.236019,0.240379,0.246266,0.250929,0.247341,0.245978,0.247048,0.246609,0.246487,0.238971,0.231700,0.229485,0.227450,0.221959,0.216337,0.214844,0.212890,0.211238,0.213290,0.209309,0.207602,0.206971,0.204077,0.

In [13]:
df_heart_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11641 entries, 0 to 11640
Columns: 188 entries, 0 to 187
dtypes: float64(188)
memory usage: 16.7 MB


In [23]:
n_train_examples = df_heart_train.shape[0]
n_test_examples = df_heart_test.shape[0]
n_total_examples = n_train_examples + n_test_examples
print(f'The test split has {n_test_examples} rows, or {n_test_examples/n_total_examples:.2%} of all examples (which represent {n_total_examples} rows).')

The test split has 2911 rows, or 20.00% of all examples (which represent 14552 rows).


# Visualization

In [13]:
pn.Column(pn.Row(base.param.add_one_hot_encoded, base.param.stand_features, base.param.test_size))

BokehModel(combine_events=True, render_bundle={'docs_json': {'38ad06bb-cc7d-416d-ac50-b63e44f0ede3': {'version…

In [20]:
import py_lib as lib
reload(lib)
import py_lib as lib

In [28]:
x_train_0, x_test_0, y_train_0, y_test_0 = lib.pre_process_ecg(
        df_heart_train,
        label_col_pos=-1,
        test_size=base.test_size,
        random_state=base.random_state_test_train_split,
        split_data=True,
)

In [29]:
y_train_0.value_counts()

187
1.0    6727
0.0    2585
Name: count, dtype: int64

In [30]:
y_train_0.shape

(9312,)

In [ ]:
# # Find events
# events = nk.events_find(data["Photosensor"], threshold_keep='below', event_conditions=condition_list)
# events

In [ ]:
# # Plot the location of event with the signals
# plot = nk.events_plot(events, data)